In [1]:
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep
from comet_ml import Experiment
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import os

plt.style.use(hep.style.CMS)


In [2]:
folder = "C:/Users/phili/Desktop/Studium/6. Semester/Semesterarbeit/dataset"

df_target = ak.from_parquet(folder+'/ttHTobb_2018_test.parquet')
df_eft_tbarlnutqq = ak.from_parquet(folder+'/ttHTobb_EFT_tbarlnutqq_2018_test.parquet')
df_eft_tbarqqtlnu = ak.from_parquet(folder+'/ttHTobb_EFT_tbarqqtlnu_2018_test.parquet')

df_eft = ak.concatenate([df_eft_tbarlnutqq, df_eft_tbarqqtlnu])

def reweight(weights_eft, weights_target):
    weights_eft.to_numpy(); weights_target.to_numpy()
    
    A = np.sum(weights_target)/np.sum(weights_eft)
        
    return A, A*weights_eft

weights_eft = df_eft.weight
weights_target = df_target.weight

A, weights_eft = reweight(weights_eft, weights_target)
df_eft = ak.with_field(df_eft, weights_eft, "weight")


print("K-factor: ", A)
"""
plt.hist(rw_weights_eft, bins=100, label="Reweighted weights EFT")
plt.hist(weights_target, bins=100, label="Reweighted weights Target")
plt.legend()
plt.show()
"""

K-factor:  0.7826885718054716


'\nplt.hist(rw_weights_eft, bins=100, label="Reweighted weights EFT")\nplt.hist(weights_target, bins=100, label="Reweighted weights Target")\nplt.legend()\nplt.show()\n'

In [3]:

def standardize_feature(col: np.ndarray, transform: str = None):
    """
    Apply optional transform to col, then z-score:
      - transform="log1p": replace col -> log(col + 1)
      - transform=None: leave col as is
    Returns the transformed, zero-mean, unit-std array.
    """
    # 1) optional transform
    if transform == "log1p":
        col = np.log(col + 1)
    # 2) compute mean & std
    mu = np.mean(col)
    sigma = np.std(col)
    # 3) z-score
    return (col - mu) / (sigma if sigma != 0 else 1.0)

def build_X_and_standardize(df_combined, objects, fields, orders):
    """
    Pull out raw arrays, standardize each (pt->log1p+z, eta->z, phi->z),
    and stack into an (N, D) array.
    """
    X_cols = []
    for obj in objects:
        for field in fields:
            for order in orders:
                # extract a 1d numpy array
                raw = getattr(getattr(df_combined, obj), field)[:, order].to_numpy()
                # pick transform type
                if field == "pt":
                    col = standardize_feature(raw, transform="log1p")
                else:
                    col = standardize_feature(raw, transform=None)
                X_cols.append(col)
    # stack into (n_events, n_features)
    return np.stack(X_cols, axis=1)

# ——————————————————————————————
# usage:

# 1) label & concat as before
df_target["label"] = 1
df_eft   ["label"] = 0
df_combined = ak.concatenate([df_target, df_eft])

# 2) define your objects/fields/orders
objects = ["top", "antitop", "lepton_gen", "lepton_reco", "higgs"]
fields  = ["pt", "eta", "phi"]
orders  = [0]
length_features = len(objects)*len(fields)*len(orders)

# 3) build standardized X
X = build_X_and_standardize(df_combined, objects, fields, orders)
y = df_combined.label.to_numpy()
w = df_combined.weight.to_numpy()

# 4) to torch
X_torch = torch.tensor(X, dtype=torch.float32)
y_torch = torch.tensor(y, dtype=torch.float32)
w_torch = torch.tensor(w, dtype=torch.float32)


In [4]:
# Initialize a Comet experiment
experiment = Experiment(
    api_key="s0G6qTP8E4YpcrewlNfry98cY",
    project_name="ttHbb-reweighting",
    workspace="philiw"
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/philiw/tthbb-reweighting/2bf3c19a766643c2b264ef05eb8e5cfb



COMET INFO: Couldn't find a Git repository in 'c:\\Users\\phili\\Desktop\\Studium\\6. Semester\\Semesterarbeit\\Binary_Classifier_Reweighting' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [5]:
class SimpleClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=64):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc_out = nn.Linear(hidden_dim, 1)  # Output = 1 dimension (logit)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        # No activation on the last layer, we'll apply sigmoid in the loss
        return self.fc_out(x)

In [6]:
n_epochs = 10  # adjust as needed
hidden_dim = 64  # hidden layer size
batch_size = 2048  # adjust as needed
learning_rate = 1e-4  # adjust as needed


# Log hyperparameters
experiment.log_parameters({
    "input_dim": length_features,
    "hidden_dim": hidden_dim,
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "n_epochs": n_epochs
})


n_events = len(X_torch)
all_idx = np.arange(n_events)

idx_train, idx_test = train_test_split(all_idx, test_size=0.5, random_state=42) # Optional: stratify

# keep  convenience copy that contains ONLY the EFT rows inside the train fold
eft_train_idx = idx_train[y[idx_train] == 0]
target_train_idx = idx_train[y[idx_train] == 1]
eft_test_idx = idx_test[y[idx_test] == 0]
target_test_idx = idx_test[y[idx_test] == 1]

X_train = X_torch[idx_train]
y_train = y_torch[idx_train]
w_train = w_torch[idx_train]

X_test = X_torch[idx_test]
y_test = y_torch[idx_test]
w_test = w_torch[idx_test]

# 2. Build train and test datasets
train_dataset = TensorDataset(X_train, y_train, w_train)
test_dataset  = TensorDataset(X_test,  y_test,  w_test)

# 3. Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

# 4. (Optional) Log sizes
experiment.log_metric("train_size", len(train_dataset))
experiment.log_metric("test_size",  len(test_dataset))

model = SimpleClassifier(input_dim=length_features, hidden_dim=hidden_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
bce_loss = nn.BCEWithLogitsLoss(reduction='none')  # We'll do weighting manually

In [7]:
def calculate_loss(model, x, y, w):
    logits = model(x)  # shape: [batch_size, 1]
    # BCE loss with optional event weight
    raw_loss = bce_loss(logits.squeeze(), y)
    # multiply by MC weights, then average
    loss = (raw_loss * w).mean()
    return loss	

def train_model(model, loader, optimizer, n_epochs):
    for epoch in range(n_epochs):
        model.train()
        epoch_loss = 0.0
        for batch_x, batch_y, batch_w in loader:
            optimizer.zero_grad()
            
            loss = calculate_loss(model, batch_x, batch_y, batch_w)
            
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        avg_loss = epoch_loss / len(loader)
        print(f"[Epoch {epoch}] average loss = {avg_loss:.7f}")
        experiment.log_metric("average_loss", avg_loss, step=epoch)

        if epoch % 5 == 0:
            # Evaluate the model on the training set every 5 epochs

            f_eft, reweighted_wts, reweighting_factors = evaluate_model_on_eft(model, X_train, y_train, w_train)
            do_plots(f_eft, reweighted_wts, reweighting_factors, eft_indices=eft_train_idx, target_indices=target_train_idx, step=epoch)

    # Save the model and log it to Comet
    torch.save(model.state_dict(), "simple_classifier.pth")
    experiment.log_model("simple_classifier", "simple_classifier.pth")

    return model

def evaluate_model_on_eft(model, X, y, w):
    model.eval()

    # Grab only EFT events from the combined dataset
    EFT_mask = y == 0
    eft_xft = X[EFT_mask]
    w_mc_eft = w[EFT_mask]

    with torch.no_grad():
        logits_eft = model(eft_xft).squeeze()
        f_eft = torch.sigmoid(logits_eft)  # f(x)
        w_new = f_eft / (1.0 - f_eft)      # w(x) = f/(1-f)

    # Final reweighted MC weights for EFT:
    reweighted_wts = w_mc_eft * w_new
    
    return f_eft, reweighted_wts, w_new

def plot_dist_f(f_eft, eft_indices = None, epoch=-1):
    masked_weights_eft = df_combined.weight[eft_indices].to_numpy()

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.hist(f_eft, bins=50, weights=masked_weights_eft)
    ax.set_xlabel("f(x)")
    ax.set_ylabel("Weighted Events")
    filename = f"distribution_f.png"
    #plt.show()  # Optionally, you can omit this in non-interactive scripts
    if epoch == -1:
        plt.savefig(filename)
        experiment.log_image(filename, name=filename)
    else:
        plt.savefig(filename)
        experiment.log_image(filename, name=filename, step=epoch)
    plt.close(fig)

def plot_dist_w(reweighting_factors, eft_indices = None, epoch=-1):
    masked_weights_eft = df_combined.weight[eft_indices].to_numpy()

    fig = plt.figure()
    ax = fig.add_subplot()
    ax.hist(reweighting_factors, bins=50, weights=masked_weights_eft)
    ax.set_xlabel("w(x)")
    ax.set_ylabel("Weighted Events")
    filename = f"distribution_w.png"
    #plt.show()  # Optionally, you can omit this in non-interactive scripts
    if epoch == -1:
        plt.savefig(filename)
        experiment.log_image(filename, name=filename)
    else:
        plt.savefig(filename)
        experiment.log_image(filename, name=filename, step=epoch)
    plt.close(fig)

def get_bins_and_scale(object, field, num_bins=50):
    if field == 'pt':
        if object == 'top' or object == 'antitop' or object == 'higgs':
            return np.logspace(1.3, 2.9, num_bins), 'log'
        if object == 'lepton_gen' or object == 'lepton_reco':
            return np.logspace(1.4, 2.7, num_bins), 'log'
    elif field == 'eta':
        if object == 'lepton_gen' or object == 'lepton_reco':
            return np.linspace(-3, 3, num_bins), 'linear'
        else:
            return np.linspace(-4, 4, num_bins), 'linear'
    elif field == 'phi':
        return np.linspace(-np.pi, np.pi, num_bins), 'linear'
    else:
        raise ValueError(f"Unknown field: {field}")

def plot_1d_dist(reweighted_wts, eft_indices=None, target_indices = None, epoch=-1): #Default select all events in 
    # Loop over each combination
    for obj in objects:
        for field in fields:

            bins, scale = get_bins_and_scale(obj, field)
            
            for order in orders:
                # Extract the appropriate variable from the EFT and Target data, select indices based on what the model evaluated on
                eft_variable = getattr(getattr(df_combined, obj), field)[:, order][eft_indices]
                target_variable = getattr(getattr(df_combined, obj), field)[:, order][target_indices]
                masked_weights_eft = df_combined.weight[eft_indices]
                masked_weights_target = df_combined.weight[target_indices]

                # Define your histogram bins (you may wish to adjust these per variable)
                bin_centers = 0.5 * (bins[1:] + bins[:-1])

                # Compute the histograms
                
                hist_eft, _    = np.histogram(eft_variable, bins=bins, weights=masked_weights_eft)
                hist_rw_eft, _ = np.histogram(eft_variable, bins=bins, weights=reweighted_wts.numpy())
                hist_target, _ = np.histogram(target_variable, bins=bins, weights=masked_weights_target)

                # Create the figure with a top and bottom panel
                fig, (ax_top, ax_bot) = plt.subplots(
                    2, 1, figsize=(10, 10),
                    gridspec_kw={'height_ratios': [5, 2]}
                )

                # Top panel: draw the histograms
                ax_top.stairs(hist_eft, bins, label="EFT data (unweighted)", fill=False)
                ax_top.stairs(hist_rw_eft, bins, label=f"Reweighted EFT ({order})", fill=False)
                ax_top.stairs(hist_target, bins, label="Target data", fill=False)
                ax_top.set_xscale(scale)
                ax_top.set_xlabel(f"{obj}.{field}")
                ax_top.set_ylabel("Weighted events")
                ax_top.set_title(f"Histogram of {obj}.{field}\n(reweighting: {order})")
                ax_top.legend()

                # Bottom panel: compute and plot the ratios
                # Ratio of target to original EFT
                ratio_old = np.where(hist_eft != 0, hist_target / hist_eft, np.nan)
                # Ratio of target to reweighted EFT
                ratio_rw = np.where(hist_rw_eft != 0, hist_target / hist_rw_eft, np.nan)

                ax_bot.plot(bin_centers, ratio_old, 'o-', label="Original ratio", markersize=3)
                ax_bot.plot(bin_centers, ratio_rw, 'o-', label=f"Reweighted ratio ({order})", markersize=3)
                ax_bot.set_xscale(scale)
                ax_bot.axhline(1.0, color='gray', linestyle='--')
                ax_bot.set_ylim(0.5, 1.5)
                ax_bot.set_ylabel("Target / EFT")
                ax_bot.set_xlabel(f"{obj}.{field}")
                ax_bot.legend(fontsize=10)

                plt.tight_layout()

                # Save and log the figure
                filename = f"1d_reweighting_{obj}_{field}_{order}.png"
                #plt.show()  # Optionally, you can omit this in non-interactive scripts
                if epoch == -1:
                    plt.savefig(filename)
                    experiment.log_image(filename, name=filename)
                else:
                    plt.savefig(filename)
                    experiment.log_image(filename, name=filename, step=epoch)
                # Close the plot to free up memory before the next loop iteration
                plt.close(fig)

def plot_2d_dist(reweighted_wts, eft_indices=None, target_indices = None, epoch=-1):
    for i, obj1 in enumerate(objects):
        for j, obj2 in enumerate(objects):
            if i >= j:
                continue
            for field1 in fields:
                for field2 in fields:
                    for order in orders:
                        # get bins for the 2D histogram
                        bins1, scale1 = get_bins_and_scale(obj1, field1, num_bins=20)
                        bins2, scale2 = get_bins_and_scale(obj2, field2, num_bins=20)

                        # extract EFT phase‐space coords
                        eft_x = getattr(getattr(df_combined, obj1), field1)[:, order][eft_indices].to_numpy()
                        eft_y = getattr(getattr(df_combined, obj2), field2)[:, order][eft_indices].to_numpy()
                        target_x = getattr(getattr(df_combined, obj1), field1)[:, order][target_indices].to_numpy()
                        target_y = getattr(getattr(df_combined, obj2), field2)[:, order][target_indices].to_numpy()
                        masked_weights_eft = df_combined.weight[eft_indices]
                        masked_weights_target = df_combined.weight[target_indices]
                        
                        # 5) make figure with one subplot per bin

                        fig, axs = plt.subplots(1, 4, figsize=(32, 8))

                        h_eft_result = axs[0].hist2d(
                            eft_x, eft_y, bins=[bins1, bins2],
                            weights=masked_weights_eft.to_numpy(), cmap='inferno'  
                        )
                        axs[0].set_xscale(scale1)
                        axs[0].set_yscale(scale2)
                        axs[0].set_title("EFT (unweighted)")
                        axs[0].set_xlabel(f"{obj1}-{field1}")
                        axs[0].set_ylabel(f"{obj2}-{field2}")
                        plt.colorbar(h_eft_result[3], ax=axs[0])
                        
                        # Reweighted EFT 2D histogram
                        h_rew_result = axs[1].hist2d(
                            eft_x, eft_y, bins=[bins1, bins2],
                            weights=reweighted_wts, cmap='inferno'
                        )
                        axs[1].set_xscale(scale1)
                        axs[1].set_yscale(scale2)
                        axs[1].set_title("Reweighted EFT")
                        axs[1].set_xlabel(f"{obj1}-{field1}")
                        axs[1].set_ylabel(f"{obj2}-{field2}")
                        plt.colorbar(h_rew_result[3], ax=axs[1])
                        
                        # Target 2D histogram
                        h_target_2d = np.histogram2d(
                            target_x, target_y, bins=[bins1, bins2],
                            weights=masked_weights_target.to_numpy()
                        )[0]
                        
                        # 2D ratio: target / EFT (original)
                        ratio_original = np.where(h_eft_result[0] != 0, h_target_2d / h_eft_result[0], np.nan)
                        h3 = axs[2].pcolormesh(
                            bins1, 
                            bins2, 
                            ratio_original.T,  # note transpose
                            cmap='inferno',
                            vmin=0.5,
                            vmax=1.5,
                            shading='auto'
                        )
                        axs[2].set_xscale(scale1)
                        axs[2].set_yscale(scale2)
                        axs[2].set_title("Ratio: Target / EFT")
                        axs[2].set_xlabel(f"{obj1}-{field1}")
                        axs[2].set_ylabel(f"{obj2}-{field2}")
                        plt.colorbar(h3, ax=axs[2])
                        
                        # 2D ratio: target / EFT (reweighted)
                        ratio_rew = np.where(h_rew_result[0] != 0, h_target_2d / h_rew_result[0], np.nan)

                        h4 = axs[3].pcolormesh(
                            bins1, 
                            bins2,        
                            ratio_rew.T, 
                            cmap='inferno',
                            vmin=0.5,
                            vmax=1.5,
                            shading='auto'
                        )
                        axs[3].set_xscale(scale1)
                        axs[3].set_yscale(scale2)
                        axs[3].set_title("Ratio: Target / Reweighted EFT")
                        axs[3].set_xlabel(f"{obj1}-{field1}")
                        axs[3].set_ylabel(f"{obj2}-{field2}")
                        plt.colorbar(h4, ax=axs[3])
                        
                        plt.tight_layout()

                        filename = f"2d_reweighting_{obj1}_{field1}_{obj2}_{field2}.png"
                        plt.savefig(filename)
                        experiment.log_image(filename, name=filename)
                        plt.close(fig)

def plot_weights_in_1d_phasespace(reweighting_factors, eft_indices=None):
    n_bins = 3
    quantiles = np.quantile(reweighting_factors, np.linspace(0, 1, n_bins + 1))
    reweighting_factors_bins = np.digitize(reweighting_factors, bins=quantiles[1:-1], right=False)
    labels = {
        i: f"{quantiles[i]:.2f}–{quantiles[i+1]:.2f}"
        for i in range(n_bins)
    }

    for i, obj in enumerate(objects):
        for field in fields:
            for order in orders:
                # get bins for the 1D histogram
                bins, scale = get_bins_and_scale(obj, field)

                # extract EFT phase‐space coords
                eft_x = getattr(getattr(df_combined, obj), field)[:, order][eft_indices].to_numpy()
                w = df_combined.weight[eft_indices].to_numpy()

                # 5) make figure with one subplot per bin
                fig, axs = plt.subplots(
                    1, n_bins,
                    figsize=(4 * n_bins, 4),
                    sharex=True, sharey=True
                )

                # 6) fill each subplot with a weighted histogram
                for bin_idx in range(n_bins):
                    mask = (reweighting_factors_bins == bin_idx)
                    h = axs[bin_idx].hist(
                        eft_x[mask],
                        bins=bins,
                        weights=w[mask],
                        histtype='stepfilled',
                        label=labels[bin_idx],
                        alpha=0.5,
                        color='blue'
                    )
                    axs[bin_idx].set_xscale(scale)
                    axs[bin_idx].set_title(labels[bin_idx])
                    if bin_idx == 0:
                        axs[0].set_xlabel(field)
                        axs[0].set_ylabel("Weighted events")

                """
                fig.suptitle(
                    f"{obj}-{field} histograms by w‐quantile",
                )
                fig.colorbar(
                    h[3], 
                    ax=axs.ravel().tolist(),
                )"""

                filename = f"1d_phasespace_hist_{obj}_{field}.png"
                plt.savefig(filename)
                experiment.log_image(filename, name=filename)
                plt.close(fig)

def plot_weights_in_2d_phasespace(reweighting_factors, eft_indices=None):
    # --- your existing quantile‐binning code ---
    n_bins = 3
    quantiles = np.quantile(reweighting_factors, np.linspace(0, 1, n_bins + 1))
    reweighting_factors_bins = np.digitize(reweighting_factors, bins=quantiles[1:-1], right=False)
    labels = {
        i: f"{quantiles[i]:.2f}–{quantiles[i+1]:.2f}"
        for i in range(n_bins)
    }

    # 3) loop over object/field combinations
    for i, obj1 in enumerate(objects):
        for j, obj2 in enumerate(objects):
            if i >= j:
                continue
            for field1 in fields:
                for field2 in fields:
                    for order in orders:
                        # get bins for the 2D histogram
                        bins1, scale1 = get_bins_and_scale(obj1, field1, num_bins=20)
                        bins2, scale2 = get_bins_and_scale(obj2, field2, num_bins=20)

                        # extract EFT phase‐space coords
                        eft_x = getattr(getattr(df_combined, obj1), field1)[:, order][eft_indices].to_numpy()
                        eft_y = getattr(getattr(df_combined, obj2), field2)[:, order][eft_indices].to_numpy()
                        w = df_combined.weight[eft_indices].to_numpy()

                        # 5) make figure with one subplot per bin
                        fig, axs = plt.subplots(
                            1, n_bins,
                            figsize=(4 * n_bins, 4),
                            sharex=True, sharey=True
                        )

                        # 6) fill each subplot with a weighted 2D histogram
                        for bin_idx in range(n_bins):
                            mask = (reweighting_factors_bins == bin_idx)
                            h = axs[bin_idx].hist2d(
                                eft_x[mask], eft_y[mask],
                                bins=[bins1, bins2],
                                weights=w[mask],
                                cmap=plt.cm.viridis
                            )
                            axs[bin_idx].set_xscale(scale1)
                            axs[bin_idx].set_yscale(scale2)
                            axs[bin_idx].set_title(labels[bin_idx])
                            if bin_idx == 0:
                                axs[0].set_xlabel(field1)
                                axs[0].set_ylabel(field2)

                        """
                        fig.suptitle(
                            f"{obj1}-{field1} vs {obj2}-{field2} 2D histograms by w‐quantile",
                        )
                        """
                        fig.colorbar(
                            h[3], 
                            ax=axs.ravel().tolist(),
                        )

                        filename = f"2d_phasespace_hist_{obj1}_{field1}_{obj2}_{field2}.png"
                        plt.savefig(filename)
                        experiment.log_image(filename, name=filename)
                        plt.close(fig)

def do_plots(f_eft, reweighted_wts, reweighting_factors, eft_indices=None, target_indices=None, step=n_epochs):
    # Plot the distribution of f(x) and w(x)
    plot_dist_f(f_eft, eft_indices, step)
    plot_dist_w(reweighting_factors, eft_indices, step)

    # Plot the 1D distributions
    plot_1d_dist(reweighted_wts, eft_indices, target_indices, step)
    plot_2d_dist(reweighted_wts, eft_indices, target_indices, step)

def train():
    # Train the model
    return  train_model(model, train_loader, optimizer, n_epochs)

def evaluate(trained_model):

    # Evaluate the model
    test_loss = calculate_loss(trained_model, X_test, y_test, w_test)
    experiment.log_metric("test_loss", test_loss)
    f_eft, reweighted_wts, reweighting_factors = evaluate_model_on_eft(trained_model, X_test, y_test, w_test)
    

    do_plots(f_eft, reweighted_wts, reweighting_factors, eft_indices=eft_test_idx, target_indices=target_test_idx)

    plot_weights_in_1d_phasespace(reweighting_factors, eft_test_idx)
    plot_weights_in_2d_phasespace(reweighting_factors, eft_test_idx)

In [8]:
trained_model = train()

[Epoch 0] average loss = 0.0013485


c:\Users\phili\AppData\Local\Programs\Python\Python312\Lib\site-packages\awkward\_nplikes\array_module.py:290: RuntimeWarning: divide by zero encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))
c:\Users\phili\AppData\Local\Programs\Python\Python312\Lib\site-packages\awkward\_nplikes\array_module.py:290: RuntimeWarning: invalid value encountered in divide
  return impl(*broadcasted_args, **(kwargs or {}))
C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:230: RuntimeWarning: divide by zero encountered in divide
  ratio_original = np.where(h_eft_result[0] != 0, h_target_2d / h_eft_result[0], np.nan)
C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:248: RuntimeWarning: divide by zero encountered in divide
  ratio_rew = np.where(h_rew_result[0] != 0, h_target_2d / h_rew_result[0], np.nan)
C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:230: RuntimeWarning: invalid value encountered in divide
  ratio_original = np.where(h_eft

[Epoch 1] average loss = 0.0013469
[Epoch 2] average loss = 0.0013465
[Epoch 3] average loss = 0.0013461
[Epoch 4] average loss = 0.0013460
[Epoch 5] average loss = 0.0013456
[Epoch 6] average loss = 0.0013454
[Epoch 7] average loss = 0.0013454
[Epoch 8] average loss = 0.0013452
[Epoch 9] average loss = 0.0013450


In [9]:
evaluate(trained_model)

experiment.end()

C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:230: RuntimeWarning: divide by zero encountered in divide
  ratio_original = np.where(h_eft_result[0] != 0, h_target_2d / h_eft_result[0], np.nan)
C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:230: RuntimeWarning: invalid value encountered in divide
  ratio_original = np.where(h_eft_result[0] != 0, h_target_2d / h_eft_result[0], np.nan)
C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:248: RuntimeWarning: divide by zero encountered in divide
  ratio_rew = np.where(h_rew_result[0] != 0, h_target_2d / h_rew_result[0], np.nan)
C:\Users\phili\AppData\Local\Temp\ipykernel_17856\3149245249.py:248: RuntimeWarning: invalid value encountered in divide
  ratio_rew = np.where(h_rew_result[0] != 0, h_target_2d / h_rew_result[0], np.nan)
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ----------------